In [30]:
import pandas as pd
import json
import os
from globals import BASE_DIR

available_datasets = ["foursquaretky", "yelp", "gowalla", "brightkite", "snowcard"]

In [36]:
for dataset in available_datasets:
    OUTPUT_DIR = f"{BASE_DIR}{dataset}_dataset/recommendations/"
    recs = os.listdir(OUTPUT_DIR)
    
    if '.DS_Store' in recs:
        recs.remove('.DS_Store')

    capri_recs = []
    for dir in recs:
        if dir.split("-")[1] == "contextpoi":  # Filter for contextpoi directories
            capri_recs.append(dir)
    
    for dir in capri_recs:
        # Get the correct directory path for the current dataset and capri_rec directory
        dir_path = os.path.join(OUTPUT_DIR, dir)
        files = os.listdir(dir_path)
        
        # Find the file that starts with 'Eval'
        eval_file = next((f for f in files if f.startswith('Eval')), None)
        
        if eval_file:
            eval_csv = os.path.join(dir_path, eval_file)  # Full path to the correct CSV file
            json_file = os.path.join(dir_path, 'general_evaluation.json')

            # Read the evaluation CSV into a pandas DataFrame
            eval_df = pd.read_csv(eval_csv)

            # Assuming the CSV has a single row with all the metrics, convert the row to a dictionary
            metrics = eval_df.iloc[0].to_dict()
            print(f"Metrics for {dir}: {metrics}")

            # Create the structure for the JSON output
            evaluation_data = {
                "test_result": {
                    "recall@10": metrics.get("recall"),
                    "ndcg@10": metrics.get("ndcg"),
                    "precision@10": metrics.get("precision"),
                    "map@10": metrics.get("map")
                }
            }

            # Write the evaluation data to a JSON file
            with open(json_file, 'w') as f:
                json.dump(evaluation_data, f, indent=4)

            print(f"Saved general_evaluation.json to {json_file}")
        else:
            print(f"No Eval file found in {dir_path}")

        # Find the file that starts with 'Rec'
        rec_file = next((f for f in files if f.startswith('Rec')), None)
        
        if rec_file:
            rec_txt = os.path.join(dir_path, rec_file)  # Full path to the Rec txt file
            json_file = os.path.join(dir_path, 'top_k_recommendations.json')

            # Create an empty dictionary to store the recommendations
            recommendations = {}

            # Read the Rec txt file
            with open(rec_txt, 'r') as f:
                for line in f:
                    # Split the line into three components (index, user_id, item_ids)
                    parts = line.strip().split("\t")
                    
                    if len(parts) == 3:
                        user_id = parts[1]  # Second column is the user_id
                        item_ids = parts[2].split(",")  # Third column is the comma-separated item_ids

                        # Convert user_id and item_ids to strings with "_x" suffix
                        user_id = f"{user_id}_x"
                        item_ids = [f"{item}_x" for item in item_ids]

                        # Add the recommendations to the dictionary
                        recommendations[user_id] = [{"item_id": item_ids}]
            
            # Write the recommendations to a JSON file
            with open(json_file, 'w') as f:
                json.dump(recommendations, f, indent=4)

            print(f"Saved top_k_recommendations.json to {json_file}")
        else:
            print(f"No Rec file found in {dir_path}")


Metrics for foursquaretky_sample-contextpoi-LORE-Sep-16-2024_09-00-00: {'precision': 0.01067, 'recall': 0.01378, 'ndcg': 0.01216, 'map': 0.00548}
Saved general_evaluation.json to /Volumes/Forster Neu/Masterarbeit Data/foursquaretky_dataset/recommendations/foursquaretky_sample-contextpoi-LORE-Sep-16-2024_09-00-00/general_evaluation.json
Saved top_k_recommendations.json to /Volumes/Forster Neu/Masterarbeit Data/foursquaretky_dataset/recommendations/foursquaretky_sample-contextpoi-LORE-Sep-16-2024_09-00-00/top_k_recommendations.json
Metrics for foursquaretky_sample-contextpoi-USG-Sep-16-2024_09-00-00: {'precision': 0.071, 'recall': 0.07067, 'ndcg': 0.08108, 'map': 0.04056}
Saved general_evaluation.json to /Volumes/Forster Neu/Masterarbeit Data/foursquaretky_dataset/recommendations/foursquaretky_sample-contextpoi-USG-Sep-16-2024_09-00-00/general_evaluation.json
Saved top_k_recommendations.json to /Volumes/Forster Neu/Masterarbeit Data/foursquaretky_dataset/recommendations/foursquaretky_sam

In [18]:
capri_recs

['foursquaretky_sample-contextpoi-LORE-Sep-16-2024_09-00-00',
 'foursquaretky_sample-contextpoi-USG-Sep-16-2024_09-00-00',
 'yelp_sample-contextpoi-LORE-Sep-16-2024_09-00-00',
 'yelp_sample-contextpoi-USG-Sep-16-2024_09-00-00',
 'gowalla_sample-contextpoi-LORE-Sep-16-2024_09-00-00',
 'gowalla_sample-contextpoi-USG-Sep-16-2024_09-00-00',
 'brightkite_sample-contextpoi-LORE-Sep-16-2024_09-00-00',
 'brightkite_sample-contextpoi-USG-Sep-16-2024_09-00-00',
 'snowcard_sample-contextpoi-LORE-Sep-16-2024_09-00-00',
 'snowcard_sample-contextpoi-USG-Sep-16-2024_09-00-00']